# Day 1

12/6/2019  
2:30-4:50 pm

In [1]:
# importing the requests library 
import requests 
import pandas as pd

In [ ]:
# Main:

# api-endpoint 
# URL = "http://echo.epa.gov/tools/web-services/loading-tool/dmr_rest_services.get_dmr_loadings"
URL = 'https://ofmpub.epa.gov/echo/dmr_rest_services.get_dmr_loadings'  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'output':'JSON',
         'p_year':' 2019',
         'p_huc':'11010001',
         'responseset': 150, # Number of rows to be retreived (#facilities*5)
         'download':'TopFacilityPounds' # This downloads each facility 5 times (1 per top pollutant)
         } # There might be a better API that does not do this repetititev.
  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 

In [ ]:
print('test')

## Note: 

I always set to 1000 rows, but this can be replaced by a smarter approach, where you first query the number of all facilities, then based on that and at the end double check if all of them was retreived or not.

In [ ]:
data['Results'].keys()

In [ ]:
len(data['Results']['TopFacilityPounds'])

In [ ]:
data['Results']['FacilityCounts']

### Report:
1. It took a lot to find the correct URL for API call. Finally I found it from https://echo.epa.gov/tools/web-services/loading-tool#/Water%20Pollution%20Search%2C%20Facility%20Reports%2C%20and%20Loading%20Calculations/get_dmr_rest_services_get_dmr_loadings. If you select the Blue "Try it out" button, then scroll down, where you will see the corresponding URL under "Request URL" section.
2. Updated Jessie on the progress.
3. Next steps are:
    - work on different existing API calls to see which ones you need to extract the required information.

# ====================================
Day 2
12/12/2019 4:36 pm - 6:48 pm

Goal: 
- Look for the relevant API's that provides the data required by project.

In [ ]:
facilities = data['Results']['TopFacilityPounds']

In [ ]:
facilities_stats = data['Results']['TopFacilityPounds']

In [ ]:
Success_flag = facility_counts = data['Results']['Message']

In [ ]:
import time


In [ ]:
# ExternalPermitNmbr = list()
# FacilityName = list()
# City = list()
# GeocodeLatitude = list()
# GeocodeLongitude = list()
# MajorMinorStatusFlag = list()
# CountyName = list()
# SicCode1 = list()
# PollutantDesc = list()
data_dict = {'ExternalPermitNmbr':[],
            'FacilityName':[],
            'City':[],
            'GeocodeLatitude':[],
            'GeocodeLongitude':[],
            'MajorMinorStatusFlag':[],
            'CountyName':[],
            'SicCode1':[],
             'zip':[],
             'FacilityType':[],
             'PermitType':[],
             'PermitEffectiveDate':[],
             'PermitExpirationDate':[],
             'ApprovedPretreatmentProgram':[],
             'FacilityDesignFlow':[],
             'AvgFacilityFlow':[],
             'SicCode':[],
             'Pollutants':[],
             'ParameterCode':[],
             'TotalPounds':[],
             'MaxAllowablePounds':[],
             'LoadOverLimit':[],
             'TotalTwpe':[],
             'MaxAllowableTwpe':[],
             'LoadOverLimitTwpe':[],
             'QcFlag':[],
             'ExceedanceCount':[],
             'QcReviewPounds':[],
             'QcReviewTwpe':[]
             
            }

facility_loading_URL = 'https://ofmpub.epa.gov/echo/dmr_rest_services.get_facility_report'  
    # defining a params dict for the parameters to be sent to the API 
    
facility_id_list = list() # just to check and eliminate the repeating facilities

for i in range(len(facilities)):
    if  (facilities[i]['ExternalPermitNmbr'] in facility_id_list)==False:
        facility_id_list.append(facilities[i]['ExternalPermitNmbr'])
        
        PARAMS = {'output':'JSON',
                 'p_permit_id':facilities[i]['ExternalPermitNmbr'],
                 'p_year':'2019',
                 } 
        status_code = 0
        j=1
        while status_code==0:
            try:
                # sending get request and saving the response as response object 
                facility_r = requests.get(url = facility_loading_URL, params = PARAMS) 
                status_code = 1
#                 print('Succ')
            except requests.exceptions.ConnectionError:
        # Does it ever gets here??????
                status_code = 0
                print('attempt {}, connection refused!'.format(j))
                j+=1
                facility_r.status_code='Connection refused'
        # extracting data in json format 
        facility_data = facility_r.json() 
        #         data_dict['Pollutants'].append(facility_data['Results']['PollutantLoads'])
        
        for x in facility_data['Results']['PollutantLoads']:
            data_dict['ExternalPermitNmbr'].append(facilities[i]['ExternalPermitNmbr'])
            data_dict['FacilityName'].append(facilities[i]['FacilityName'])
            data_dict['City'].append(facilities[i]['City'])
            data_dict['GeocodeLatitude'].append(facilities[i]['GeocodeLatitude'])
            data_dict['GeocodeLongitude'].append(facilities[i]['GeocodeLongitude'])
            data_dict['MajorMinorStatusFlag'].append(facilities[i]['MajorMinorStatusFlag'])
            data_dict['CountyName'].append(facilities[i]['CountyName'])
            data_dict['SicCode1'].append(facilities[i]['SicCode1'])
            
            data_dict['zip'].append(facility_data['Results']['FacilityInfo']['FacZip'])
            data_dict['FacilityType'].append(facility_data['Results']['FacilityInfo']['FacilityType'])
            data_dict['PermitType'].append(facility_data['Results']['FacilityInfo']['PermitType'])
            data_dict['PermitEffectiveDate'].append(facility_data['Results']['FacilityInfo']['PermitEffectiveDate'])
            data_dict['PermitExpirationDate'].append(facility_data['Results']['FacilityInfo']['PermitExpirationDate'])
            data_dict['ApprovedPretreatmentProgram'].append(facility_data['Results']['FacilityInfo']['PretreatProgram'])
            data_dict['FacilityDesignFlow'].append(facility_data['Results']['FacilityInfo']['FacilityDesignFlow'])
            data_dict['AvgFacilityFlow'].append(facility_data['Results']['FacilityInfo']['AvgFacilityFlow'])
            data_dict['SicCode'].append(facility_data['Results']['FacilityInfo']['SicCode'])
         
            data_dict['Pollutants'].append(x['PollutantName'])
            data_dict['ParameterCode'].append(x['ParameterCode'])
            data_dict['TotalPounds'].append(x['TotalPounds'])
            data_dict['MaxAllowablePounds'].append(x['MaxAllowablePounds'])
            data_dict['LoadOverLimit'].append(x['LoadOverLimit'])
            data_dict['TotalTwpe'].append(x['TotalTwpe'])
            data_dict['MaxAllowableTwpe'].append(x['MaxAllowableTwpe'])
            data_dict['LoadOverLimitTwpe'].append(x['LoadOverLimitTwpe'])
            data_dict['QcFlag'].append(x['QcFlag'])
            data_dict['ExceedanceCount'].append(x['ExceedanceCount'])
            data_dict['QcReviewPounds'].append(x['QcReviewPounds'])
            data_dict['QcReviewTwpe'].append(x['QcReviewTwpe'])


        print('{}/{}, {},     facility name: {}'.format(i+1, len(facilities),
                                                        facility_data['Results']['Message'],
                                                        facilities[i]['FacilityName']))
        time.sleep(0.5)
  

In [ ]:
df=pd.DataFrame(data_dict)

In [ ]:
df.head(20)

### Achievements at the end of day 2:


1. Decoded the fields of json file
2. Retrived a lot of fields of data using API.
3. At the moment I am saving first as a dictionary, then convert it to a dataframe.


### Things to do on day 3:
1. Identify list of existing and missing data
2. Work on retriving the missing data, maybe using some extra API calls about the specific polutant or the specific facility
3. Code optimization, specially is there a better way than using dictionary+df?
4. Populate the year and ID number columns of df.
5. Combine the facility name with city name and figure out what is the trailing number? Is it post code?
6. Work on the name of columns of df to match it with column names in Database Template.

# Day 3
12/19/2019  

11:46 am - 12:40 pm

4:00 pm - 7:00 pm


List of missing columns:
- Facility type (example: POTW)
- Permit type (example: NPDES Individual Permit)
- Permit Effective Date
- Permit Expiration Date
- Approved Pretreatment Program
- Facility Design Flow (Permit Application) (MGD)
- Parameter code
- MAx Allowable load
- Max Allowable TWPE (lb-eq/yr)
- TWPE overlimit
- Contains potential Outliers?
- Number of Exceedances

I think majority of these missing columns relate to facility information. So lets query the information about each facility.

## Achievements on day 3: 

**Major progress**

- Retreived almost all of the required fields in the proper format:
    - There is multiple rows per facility corresponding to each pollutant
    
    
    
    


## Plan for Day 4:
 1. Look again to columns of the dataframe and look for missing fields
 2. If no missing field, then consider possible issues:
     - I am using try-except inside a while loop to catch the unresponsiveness from server. Not sure if it catches anything at all! Make sure about this.
     - The main (outer) query that retreives all facilities per watershed: I set the number of rows to 1000. It has a facility-stats that shows all facilities availablae in this watershed. Make sure at the end that the data for all of them are retreived.
 3. Next step will be to add another outer call that calls all the watersheds. (So far I ahve been calling only one watershed.). This can be done with for loop that or one call with multiple watershed codes in it. Think about this, which one is better.
 4. Finally you need to add another layer where you retreive data from different years. You can not provide multiple years within one API call. So this definitely needs separate calls for each year.